- node2vec 라이브러리는 'pip install node2vec' 입력하면 설치할 수 있습니다.
- 라이브러리 출처: https://github.com/eliorc/node2vec
- node2vec 논문: https://arxiv.org/abs/1607.00653
- deepwalk 논문: https://arxiv.org/abs/1403.6652

In [31]:
import pandas as pd
import numpy as np
from node2vec import Node2Vec
import networkx as nx
import warnings; warnings.filterwarnings('ignore')

In [2]:
elist = pd.read_csv('space_adj_v3.csv', header=None)

In [5]:
elist = elist.iloc[:, :4]
elist = elist.rename(columns={0:'s1', 1:'s1_label', 2:'s2', 3:'s2_label'})
elist = elist[['s1', 's2']]; elist.head()

,s1,s2
0,99248,718026
1,99308,913844
2,99368,913844
3,99428,87991
4,99428,913844


- node2vec 라이브러리는 networkx 형식의 그래프를 input으로 받으므로 edgelist를 networkx 그래프 형태로 변환.

In [12]:
G = nx.from_pandas_edgelist(elist, 's1', 's2')

- node2vec 파라미터 설정

In [17]:
emb_dim = 16
walk_len = 10
num_walk = 10
window_size = 3
n2v_epoch = 1
n_iter = 5

- node2vec 임베딩 수행

In [20]:
model = Node2Vec(G, dimensions=emb_dim, walk_length=walk_len, num_walks=num_walk)
n2v = model.fit(window=window_size, min_count=1)
node_emb = n2v.wv

Computing transition probabilities:   0%|          | 0/16090 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|███████████████████████████████████████████████████████| 10/10 [01:27<00:00,  8.71s/it]


- 임베딩을 데이터프레임으로 표시
- 각 부재/공간의 임베딩이 지정한 dimension(emb_dim) 만큼의 데이터로 표현된다.

In [33]:
el_ids = pd.DataFrame(node_emb.wv.vocab.keys(), columns=['id'])
n2v_df = pd.concat([el_ids, pd.DataFrame(node_emb.wv.vectors)], axis=1); n2v_df.iloc[:5]

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,209047,-0.928066,-1.027372,-0.437074,-0.180518,0.622333,-1.341922,-0.204972,-1.610391,-0.483020,-0.830790,-0.478598,0.811483,0.475787,-0.346786,0.410559,0.139909
1,1013773,-0.356924,0.727578,-0.637722,0.399923,0.302692,-1.765869,-1.476964,0.353335,0.262062,-0.910535,-0.137551,1.114366,0.845917,-0.279772,-0.224211,0.018927
2,209251,-0.404510,0.769772,-0.614292,0.340548,0.309851,-1.795105,-1.454815,0.403340,0.296095,-0.973663,-0.097357,1.071220,0.861355,-0.397289,-0.183062,0.031031
3,510,0.948471,0.642889,-0.333743,-0.637554,0.510484,-0.352347,0.384359,-0.280178,-0.901641,-0.912026,-2.414509,0.339611,1.120640,-0.205278,0.549256,0.815407
4,1005120,0.765668,-0.141117,-1.144521,0.116728,2.077120,-0.366280,-0.097086,0.262583,0.020530,-0.236509,-1.022013,0.080896,1.469711,-0.366343,1.720756,-0.251622
